# Use Mistral Nemo for dialogue and embed dialogue in image

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import subprocess
import matplotlib.pyplot as plt
import textwrap
# def embedding(image_paths, prompt):
def embed_text_as_image_novel(image_path, text):
    # Open the image
    image = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(image)

    # Set up the font and size
    try:
        font = ImageFont.truetype("arial.ttf", 20)  # You can customize the font and size
    except IOError:
        font = ImageFont.load_default()

    # Wrap the text to fit the image width
    max_width = image.width - 20  # Leave some padding
    wrapped_text = textwrap.fill(text, width=40)  # Adjust width as needed

    # Calculate text size using textbbox
    text_bbox = draw.textbbox((0, 0), wrapped_text, font=font)
    text_width, text_height = text_bbox[2] - text_bbox[0], text_bbox[3] - text_bbox[1]

    # Calculate text position at the bottom of the image
    padding = 10
    text_position = (10, image.height - text_height - padding)  # Adjust padding if needed

    # Draw a rectangle behind the text for better visibility (optional)
    rectangle_bbox = [text_position[0] - 5, text_position[1] - 5, 
                    text_position[0] + text_width + 5, text_position[1] + text_height + 5]
    draw.rectangle(rectangle_bbox, fill="black")

    # Draw the text on the image
    draw.text(text_position, wrapped_text, font=font, fill="white")

    # Save the image with the embedded text
    output_path = "output_" + image_path.split('/')[-1]  # Prepend 'output_' to the filename
    image.save(output_path)
    return output_path

def display_image(image_path):
    # Open the image using PIL
    image = Image.open(image_path)

    # Display the image using matplotlib
    plt.figure(figsize=(8, 6))
    plt.imshow(image)
    plt.axis('off')  # Hide the axis
    plt.show()

def generate_text_with_mistral(prompt, image_path):
    try:
        # Call the CLI command, adjust as per the actual command structure
        command = ["ollama", "run", "mistral-nemo", image_path]
        result = subprocess.run(
            command, input=prompt,
            capture_output=True,
            text=True
        )
        
        # Check for errors
        if result.returncode != 0:
            raise Exception(f"Error generating text: {result.stderr}")
        
        return result.stdout.strip()

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

def process_images_sequentially(image_paths, prompt):
    comic_data = []
    
# for i, image_path in enumerate(image_paths):
    # Generate the dialogue for each image sequentially using the same prompt
    generated_text = generate_text_with_mistral(prompt, image_paths)
    
    # Store the image and its corresponding dialogue in the dictionary
    
    
    # Embed the dialogue into the image
    output_image_path = embed_text_as_image_novel(image_paths, generated_text)
    comic_data.append(output_image_path)
    print(f"Generated Text for {image_paths}: {generated_text}")
    print(f"Output Image Path: {output_image_path}")

    # Display the image
    display_image(output_image_path)

        # return comic_data
    

# Example usage
desc = "Scientist Tom makes a significant discovery in biology lab."

prompt = f"I want you to follow the scientific description {desc} for the image i shall provide. I want you to Generate JUST ONE small dialogue WITH THE PERSON SPEAKING STORY WISE for image and no other text."

process_images_sequentially("output.png", prompt)
